In [46]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import os
import pymysql as ps
import time


class GooSpider():
    def __init__(self, query_sting):
        self.query_sting = query_sting
        self.headers = {
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
        }
        self.url = 'https://scholar.google.com.hk/scholar'

    def _get_response(self):
        

        query_generator = {
            "hl": "zh-CN",
            "q": self.query_sting,
            "btnG": "",
            "lr": ""
        }
        response = requests.get(self.url, headers=self.headers, params=query_generator)  # proxies=proxies
        html = response.text
        return html

    def get_articles(self):
        soup = BeautifulSoup(self._get_response(), "lxml")
        articles = []
        for item in soup.find_all('div', class_='gs_r'):
            article_title = item.h3.get_text().replace('[HTML][HTML] ', '')
            ref_item = item.find('div', class_='gs_ggsd')
            article_ref = ref_item.a.get('href') if ref_item else 'null'
            citation = ''
            article_list = [datetime.now(), article_title, article_ref, self.query_sting, citation]
            articles.append(article_list)
        #print(articles)
        return articles
    def get_pdf(self):
        #print(self.get_articles()[0][2])
        for i,li in enumerate(self.get_articles()):
            
            if 'pdf' in li[2]:
                try:
                    find_pdf = requests.get(li[2], headers=self.headers, verify = False).content 
                    pdf_name = li[1][0:20]
                    print(li[2])
                    root = 'D://'
                    path = root + pdf_name
                    time.sleep(10)
                    with open(path+'.pdf', 'wb') as f:
                        f.write(find_pdf)
                        print('PDF保存成功')
                        f.close()
                except:
                    print('有点问题')
            
        

def insert_to_db(articles):
    db_config = {
        'host': '10.10.172.41',
        'port': 3306,
        'user': 'xj',
        'password': 'xj3d',
        'db': 'dspdw_dev',
        'charset': 'utf8'
    }
    db = ps.connect(**db_config)
    cursor = db.cursor()
    print('数据库连接成功')
    insert_to_article = "INSERT INTO article VALUES(%s, %s, %s,%s, %s)"
    try:
        cursor.executemany(insert_to_article, articles)
        db.commit()
        print('数据导入成功')
    except:
        db.rollback()
        print('不知道哪儿出错')
    cursor.close()
    db.close()


goo = GooSpider('egfr')
articles = goo.get_pdf()

C:\ProgramData\Anaconda3\envs\scrapy\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


https://pdfs.semanticscholar.org/4aec/d42e6407069abfe4e882011b78c2a19a0053.pdf
PDF保存成功
http://dns2.asia.edu.tw/~ysho/YSHO-English/1000%20China%20(Independent)/PDF/Lan%20Onc12,%20735.pdf
PDF保存成功


In [23]:
insert_to_db(articles)

数据库连接成功
数据导入成功
